In [1]:
import numpy as np #only library imported

def magVec(arr): #finded the magnitude of a vector with x y z
    return float((np.sqrt(arr[0]**2 + arr[1]**2 + arr[2]**2)))

def dec_dms_to_deg(deg,arcmin,arcsec): #converting degrees arcmins arcseconds into a decimal value
    sign = np.copysign(1,deg) #separate sign
    deg = (arcsec/3600) + (arcmin/60)+ np.abs(deg) #using absolute for degrees and converting to decimal for arcs
    return (sign*deg)

def ra_to_deg(hour, mins, sec): #hours mins seconds to decimal RA 
    time = (sec/3600) + (mins/60)+ hour
    total = time*360/24 #hours to degrees 
    if total >= 360:
        total = total-360
    return(total)

def percentError(calc,right):
    return((abs(right-calc)/right)*100)

def angmom(r,v): #angular momentum
    r=np.array(r)
    v = np.array(v)
    #print(v)
    ans = np.cross(r,v)
    for i in range(len(ans)):
        return(list(np.round(ans,6)))
    
def angle(sin,cos):
    if sin<0 and cos>0:
        return(-1*np.arccos(cos)+2*np.pi)
    elif sin>0 and cos<0:
        return(np.pi-np.arcsin(sin))
    elif sin<0 and cos<0:
        return(-1*np.arcsin(sin)+np.pi)
    else:
        return(np.arccos(cos))
    
def fg(tau,r2,r2dot):
#computing f and g values from od code using tau (Gaussian time), the r2 vector(x y z) and r2dot i.e. v (x y z)
    r2mag = magVec(r2) #finding magnitude of r2 vector
    g = tau - tau**3/(6*r2mag**3) + (np.dot(r2,r2dot)*tau**4)/(4*r2mag**5) #g computation
    
    part1 = 1 - tau**2/(2*r2mag**3) + (np.dot(r2,r2dot)*tau**3)/(2*r2mag**5) #f is split into 3 parts
    part2 = 3*((np.dot(r2dot, r2dot)/(r2mag**2)) - 1/r2mag**3)
    part3 = -15*(np.dot(r2dot, r2)/(r2mag**2))**2 + 1/(r2mag**3)
    f = part1 + (tau**4/(24*r2mag**3))*(part2 + part3)
    return (f,g)

def rhocomputation (a1,a3): #determining steps F and G from a1 and a3
    D1 = []
    D2 = []
    D3 = []
    
    Do = np.dot(phat1,np.cross(phat2,phat3))

    for i in range(3):
    #d1i,d2i,d3i where i = j-1
        D1.append(np.dot(np.cross(R[i],phat2),phat3)) 
        #computing dot/cross products to find the rho vectors from rho hats
        D2.append(np.dot(np.cross(phat1,R[i]),phat3))
        D3.append(np.dot(phat1,np.cross(phat2,R[i])))

    p1 = (a1*D1[0] - D1[1] + a3*D1[2])/(a1*Do)
    p2 = (a1*D2[0] - D2[1] + a3*D2[2])/(-1*Do)
    p3 = (a1*D3[0] - D3[1] + a3*D3[2])/(a3*Do)
    #r1 = rho1*rho1hat - R1(Sun-Earth value from input)
    #keep in mind rhohat is phat
    r2 = [p2*phat2[0]-R[1][0], p2*phat2[1]-R[1][1], p2*phat2[2]-R[1][2]]
    r1 = [p1*phat1[0]-R[0][0], p1*phat1[1]-R[0][1], p1*phat1[2]-R[0][2]]
    r3 = [p3*phat3[0]-R[2][0], p3*phat3[1]-R[2][1], p3*phat3[2]-R[2][2]]
    return(p1,p2,p3,r1,r2,r3)

In [3]:

with open('testcase2.csv') as file:
    entireArray = file.readlines()
    #print(entireArray)

    for i in range(1, len(entireArray)):
        entireArray[i] = entireArray[i].split(',')
        
    t1 = float(entireArray[1][0])
    t2 = float(entireArray[2][0])
    t3 = float(entireArray[3][0]) #R values rounding off
    R1 = np.array([np.float64(entireArray[1][3]), np.float64(entireArray[1][4]), np.float64(entireArray[1][5])])
    R2 = np.array([np.float64(entireArray[2][3]), np.float64(entireArray[2][4]), np.float64(entireArray[2][5])])
    R3 = np.array([np.float64(entireArray[3][3]), np.float64(entireArray[3][4]), np.float64(entireArray[3][5])])
    R = np.array([R1,R2,R3])
    ra1 = entireArray[1][1].split(':')
    ra2 = entireArray[2][1].split(':')
    ra3 = entireArray[3][1].split(':')
    
    ra1 = ra_to_deg(float(ra1[0]),float(ra1[1]),float(ra1[2]) )
    ra2 = ra_to_deg(float(ra2[0]),float(ra2[1]),float(ra2[2]) )
    ra3 = ra_to_deg(float(ra3[0]),float(ra3[1]),float(ra3[2]) )
    
    da1 = entireArray[1][2].split(':')
    da2 = entireArray[2][2].split(':')
    da3 = entireArray[3][2].split(':')
    
    da1 = dec_dms_to_deg(float(da1[0]), float(da1[1]), float(da1[2]))
    da2 = dec_dms_to_deg(float(da2[0]), float(da2[1]), float(da2[2]))
    da3 = dec_dms_to_deg(float(da3[0]), float(da3[1]), float(da3[2]))

In [ ]:
# Variable Names
# t1,t2,t3 represent the times of images in Julian Days
# ra1,ra2,ra3 and da1,da2, da3 represent Right Ascension and Declination
# R1,R2,R3 represent the Sun Vectors in AU (x y z)
# tau represent Gaussian conversion values
# phat is the rho hat vectors for the three times
# a1 and a3 represent r2 = a1*r1 + a3*r3
# r1, r2 and r3 represent the position (x y z)


# t1 = 2460118.708333333
# t2 = 2460125.708333333
# t3 = 2460132.708333333

# R1 = np.array([-2.112490042442325E-02, 9.323243076024456E-01, 4.041173475275137E-01])
# R2 = np.array([-1.391482133320911E-01,9.239895929637317E-01, 4.005002424542735E-01])
# R3 = np.array([-2.552179930127938E-01, 9.029675065304688E-01, 3.913862959375257E-01])
# R = np.array([R1,R2,R3])
# ra1 = ra_to_deg(15,48,6.22)
# ra2 = ra_to_deg(15,50,18.27)
# ra3 = ra_to_deg(15,57,23.59)

# da1 = dec_dms_to_deg(-2,46,44.3)
# da2 = dec_dms_to_deg(1,12,55.0)
# da3 = dec_dms_to_deg(6,7,24.6)

#Step C i.e. topocentric values from RA and dec to find Rho Hat
phat1 = np.array([np.cos(np.radians(ra1))*np.cos(np.radians(da1)), np.sin(np.radians(ra1))*np.cos(np.radians(da1)), np.sin(np.radians(da1))])
phat2 = np.array([np.cos(np.radians(ra2))*np.cos(np.radians(da2)), np.sin(np.radians(ra2))*np.cos(np.radians(da2)), np.sin(np.radians(da2))])
phat3 = np.array([np.cos(np.radians(ra3))*np.cos(np.radians(da3)), np.sin(np.radians(ra3))*np.cos(np.radians(da3)), np.sin(np.radians(da3))])

#Step D - Gaussian time intervals
k = 0.0172020989484 # tau and t difference constant

tau1 = k*(t1-t2)
tau0 = k*(t3-t1)
tau3 = k*(t3-t2)

#Step E - a1 and a3 computation from r2 = a1*r1 + a3*r3
a1 = tau3/tau0
a3 = -1*tau1/tau0

#Step F - calculating rho vectors from rho hat, a1, a3 and R values
p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3)
r1 = np.array(r1)
r2 = np.array(r2)
r3 = np.array(r3)

#Step G - determining crummy estimates of position vectors in AU
denom = k*(t2-t1)
v12 = (r2-r1)/denom
denom = k*(t3-t2)
v23 = (r2-r1)/denom

#finding v23 and v12 to determine v2 crummy estimates
v2 = np.array([])
v2 = (k*(t3-t2)*v12 + k*(t2-t1)*v23)/(k*(t3-t1))

#Step H - find initial f and g values by calling function
f1,g1 = fg(tau1, r2,v2)
f3,g3 = fg(tau3,r2,v2)

#Step I - recalculate new a1 and a3 values
a1 = g3/(f1*g3 - f3*g1)
a3 = -g1/(f1*g3 - f3*g1)

#r2lis keeps track of all r2 values
r2lis = []
r2lis.append(r2)

#Main Loop

#two values of r2 are computed to start the while loop comparison

#finding r1,r2 and r3 from Step 1 and 2 loop
p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3)

#conversion to numpy array
r1 = np.array(r1)
r2 = np.array(r2)
r3 = np.array(r3)

#find new r2 and r2dot value from Step 3
r2 = (g3*r1 - g1*r3)/(f1*g3 - f3*g1)
r2dot = (f3*r1 - f1*r3)/(f3*g1 - f1*g3)

#recomputing new f and g values
f1,g1 = fg(tau1, r2,r2dot)
f3,g3 = fg(tau3,r2,r2dot)

#new a1 and a3 values
a1 = g3/(f1*g3 - f3*g1)
a3 = -g1/(f1*g3 - f3*g1)

r2lis.append(r2)
count = 1
c = 173.144643267

while abs((magVec(r2lis[count-1])-magVec(r2lis[count]))/magVec(r2lis[count-1])) >= 1e-10:
    p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3) #Does Step 1 and 2 of loop
    r1 = np.array(r1)
    r2 = np.array(r2)
    r3 = np.array(r3)
    #recalculate the r1,r2 and r3 repeatedly
    r2 = (g3*r1 - g1*r3)/(f1*g3 - f3*g1)
    r2dot = (f3*r1 - f1*r3)/(f3*g1 - f1*g3)
    
    t1ac = t1 - p1/c
    t2ac = t2 - p2/c
    t3ac = t3 - p3/c
    tau1 = k*(t1ac - t2ac)
    tau3 = k*(t3ac - t2ac)
    
    f1,g1 = fg(tau1, r2,r2dot)
    f3,g3 = fg(tau3,r2,r2dot)


    a1 = g3/(f1*g3 - f3*g1)
    a3 = -g1/(f1*g3 - f3*g1)
    r2lis.append(r2)
    count+=1
    #print(count)
vvec = np.copy(r2dot)
rvec = np.copy(r2lis[-1])
print(rvec)

#equatorial to ecliptic
e = -23.4392911*np.pi/180
emat = np.array(([1,0,0],
                 [0,np.cos(e),-1*np.sin(e)],
                [0,np.sin(e),np.cos(e)]))

rvec_f = np.matmul(emat,rvec)
vvec_f = np.matmul(emat,vvec)
#print(rvec_f,vvec_f)
print("RVec is:", rvec_f)
print("VVec is:", vvec_f)

#calling orbital elements
orbit(rvec_f,vvec_f,t2)

In [ ]:
# #Variable naming
# r = position at time 2 i.e. middle, v = velocity at time 2 in x y z and AU
# a = semi-major axis
# h = angular momentum which is computed by a function
# e = eccentricity
# i = inclination
# bigo = longitude of ascending node
# smallo = argument of perihelion
# E = eccentric anomaly
# M = mean anomaly
# T = time of perihelion passage

def orbit(r,v,t):
    #semi-major axis
    rad = magVec(r)
    vec = magVec(v)
    a = 1/(2/rad-(vec)**2) #mu is 1 since in Gaussian units

    #eccentricity
    h = angmom(r,v)
    e = np.sqrt(1 - ((magVec(h)**2)/a))
    
    #inclination
    i = np.arccos(h[2]/magVec(h))*180/np.pi    
    
    #longitude of ascending node - bigO
    sin = h[0]/(magVec(h)*np.sin(i*np.pi/180))
    cos = -h[1]/magVec(h)*np.sin(i*np.pi/180)
    bigo = angle(sin,cos)*180/np.pi
    # need a quadrant check
    
    #argument of perihelion
    #smallo = U - v
    
    #finding U
    sin = r[2]/(rad*np.sin(i*np.pi/180))
    cos = (r[0]*np.cos(bigo*np.pi/180)+r[1]*np.sin(bigo*np.pi/180))/rad
    U = angle(sin,cos)*180/np.pi #needs

    #finding v
    cos = (1/e)*((a*(1-e**2)/rad)-1)
    sin = a*(1-e**2)*np.dot(r,v)/(magVec(h)*e*rad)
    nu = angle(sin,cos)*180/np.pi
    smallo = U-nu+360
    #OD3
    #finding E,M,T

    sin = (magVec(r)*np.sin(nu*np.pi/180))/(a*np.sqrt(1-(e**2)))
    cos = (a*e + magVec(r)*np.cos(nu*np.pi/180))/a
    E = angle(sin,cos)
    M = E - e*np.sin(E)
    print(M*180/np.pi)
    n = 0.01720209895/(a**(3/2))
    T = t - (M/n)
    return("semi-major axis:", a,"eccentricity:", e,"inclination:",i,"longitude of ascending node:",bigo,"argument of perihelion:",smallo,"time of perihelion passage:",T)

In [ ]:
# Variable Names
# t1,t2,t3 represent the times of images in Julian Days
# ra1,ra2,ra3 and da1,da2, da3 represent Right Ascension and Declination
# R1,R2,R3 represent the Sun Vectors in AU (x y z)
# tau represent Gaussian conversion values
# phat is the rho hat vectors for the three times
# a1 and a3 represent r2 = a1*r1 + a3*r3
# r1, r2 and r3 represent the position (x y z)


t1 = 2460495.782350000
t2 = 2460501.768870000000
t3 = 2460504.76054000000

R1 = np.array([-0.2192833281226325, 0.910891454141806, 0.3948631126793944])
R2 = np.array([-0.3169419234569733, 0.8862913016246026, 0.3841987372913946])
R3 = np.array([-0.3645923853322194, 0.8706202259365781, 0.3774039709468492])
R = np.array([R1,R2,R3])
ra1 = 283.4710833

da1 = 9.6793333
da2 = 10.5729167
da3 = 10.9134167

#Step C i.e. topocentric values from RA and dec to find Rho Hat
phat1 = np.array([np.cos(np.radians(ra1))*np.cos(np.radians(da1)), np.sin(np.radians(ra1))*np.cos(np.radians(da1)), np.sin(np.radians(da1))])
phat2 = np.array([np.cos(np.radians(ra2))*np.cos(np.radians(da2)), np.sin(np.radians(ra2))*np.cos(np.radians(da2)), np.sin(np.radians(da2))])
phat3 = np.array([np.cos(np.radians(ra3))*np.cos(np.radians(da3)), np.sin(np.radians(ra3))*np.cos(np.radians(da3)), np.sin(np.radians(da3))])

print("Rho hat", phat1,phat2,phat3)
#Step D - Gaussian time intervals
k = 0.0172020989484 # tau and t difference constant

tau1 = k*(t1-t2)
tau0 = k*(t3-t1)
tau3 = k*(t3-t2)

#Step E - a1 and a3 computation from r2 = a1*r1 + a3*r3
a1 = tau3/tau0
a3 = -1*tau1/tau0

#Step F - calculating rho vectors from rho hat, a1, a3 and R values
p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3)
print("vectors", p1)
r1 = np.array(r1)
r2 = np.array(r2)
r3 = np.array(r3)

#Step G - determining crummy estimates of position vectors in AU
denom = k*(t2-t1)
v12 = (r2-r1)/denom
denom = k*(t3-t2)
v23 = (r2-r1)/denom

#finding v23 and v12 to determine v2 crummy estimates
v2 = np.array([])
v2 = (k*(t3-t2)*v12 + k*(t2-t1)*v23)/(k*(t3-t1))

#Step H - find initial f and g values by calling function
f1,g1 = fg(tau1, r2,v2)
f3,g3 = fg(tau3,r2,v2)

#Step I - recalculate new a1 and a3 values
a1 = g3/(f1*g3 - f3*g1)
a3 = -g1/(f1*g3 - f3*g1)

#r2lis keeps track of all r2 values
r2lis = [0]
r2lis.append(r2)

#Main Loop

#two values of r2 are computed to start the while loop comparison

#finding r1,r2 and r3 from Step 1 and 2 loop
p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3)

#conversion to numpy array
r1 = np.array(r1)
r2 = np.array(r2)
r3 = np.array(r3)
print
#find new r2 and r2dot value from Step 3
r2 = (g3*r1 - g1*r3)/(f1*g3 - f3*g1)
r2dot = (f3*r1 - f1*r3)/(f3*g1 - f1*g3)

#recomputing new f and g values
f1,g1 = fg(tau1, r2,r2dot)
f3,g3 = fg(tau3,r2,r2dot)

#new a1 and a3 values
a1 = g3/(f1*g3 - f3*g1)
a3 = -g1/(f1*g3 - f3*g1)

r2lis.append(r2)
count = 2
c = 173.144643267

while abs((magVec(r2lis[count-1])-magVec(r2lis[count]))/magVec(r2lis[count-1])) >= 1e-10:
    p1,p2,p3,r1,r2,r3 = rhocomputation(a1,a3) #Does Step 1 and 2 of loop
    r1 = np.array(r1)
    r2 = np.array(r2)
    r3 = np.array(r3)
    #recalculate the r1,r2 and r3 repeatedly
    r2 = (g3*r1 - g1*r3)/(f1*g3 - f3*g1)
    r2dot = (f3*r1 - f1*r3)/(f3*g1 - f1*g3)
    t1ac = t1 - p1/c
    t2ac = t2 - p2/c
    t3ac = t3 - p3/c
    tau1 = k*(t1ac - t2ac)
    tau3 = k*(t3ac - t2ac)
    
    f1,g1 = fg(tau1, r2,r2dot)
    f3,g3 = fg(tau3,r2,r2dot)

    a1 = g3/(f1*g3 - f3*g1)
    a3 = -g1/(f1*g3 - f3*g1)
    r2lis.append(r2)
    count+=1
    if count>1000:
        break
    #print(count)
vvec = np.copy(r2dot)
rvec = np.copy(r2lis[-1])
print(rvec)

#equatorial to ecliptic
e = -23.4392911*np.pi/180
emat = np.array(([1,0,0],
                 [0,np.cos(e),-1*np.sin(e)],
                [0,np.sin(e),np.cos(e)]))

rvec_f = np.matmul(emat,rvec)
vvec_f = np.matmul(emat,vvec)
#print(rvec_f,vvec_f)
print("RVec is:", rvec_f)
print("VVec is:", vvec_f)

#calling orbital elements
orbit(rvec_f,vvec_f,t2)

In [ ]:
EPOCH=  2458187.5 ! 2018-Mar-10.00 (TDB)         Residual RMS= .22887
   EC= .1475361679263184   QR= 1.628362730803358   TP= 2457823.8683993439
   OM= 234.2781795271953   W=  151.0238479962028   IN= 20.46439527511626
   A= 1.910183950962758    MA= 135.7540840158917   ADIST= 2.192005171122157
   PER= 2.6401             N= .373328621           ANGMOM= .023514742
   DAN= 2.14552            DDN= 1.655              L= 26.8571392
   B= 9.7514468            MOID= .65270501         TP= 2017-Mar-11.3683993439
 

In [ ]:
def dec_dms_to_deg(deg,arcmin,arcsec): #converting degrees arcmins arcseconds into a decimal value
    sign = np.copysign(1,deg) #separate sign
    deg = (arcsec/3600) + (arcmin/60)+ np.abs(deg) #using absolute for degrees and converting to decimal for arcs
    return (sign*deg)

def ra_to_deg(hour, mins, sec): #hours mins seconds to decimal RA 
    time = (sec/3600) + (mins/60)+ hour
    total = time*360/24 #hours to degrees 
    if total >= 360:
        total = total-360
    return(total)

print(dec_dms_to_deg(7,21,50.1))
print(dec_dms_to_deg(7,21,51.3))
print(dec_dms_to_deg(7,21,51.3))

print(dec_dms_to_deg(7,50,20.8))
print(dec_dms_to_deg(7,50,26.0))
print(dec_dms_to_deg(7,50,32.8))

print(dec_dms_to_deg(9,40,41.6))
print(dec_dms_to_deg(9,40,45.6))
print(dec_dms_to_deg(9,40,46.9))

print(dec_dms_to_deg(10,34,18.8))
print(dec_dms_to_deg(10,34,22.5))
print(dec_dms_to_deg(10,34,23.7))

print(dec_dms_to_deg(10,54,47.1))
print(dec_dms_to_deg(10,54,48.3))
print(dec_dms_to_deg(10,54,48.1))